## CREACIÓN DEL MODELO (Calculo de probabilidades en un archivo CSV)

### Importación de librerias

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Word Cloud
from wordcloud import WordCloud
# from textacy import preprocessing
from nltk.stem.snowball import SnowballStemmer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from collections import defaultdict

### Separación del conjunto de entrenamiento y del conjunto de pruebas

In [9]:
test_size_average = 0.2

In [10]:
# Load Data 
df = pd.read_csv('./text.csv')
# Rename Columns 
df.rename(columns={'text': 'Text', 'label': 'Label'}, inplace=True)
# Rename {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
df['Label'] = df['Label'].replace(0,'Sadness')
df['Label'] = df['Label'].replace(1,'Joy')
df['Label'] = df['Label'].replace(2,'Love')
df['Label'] = df['Label'].replace(3,'Anger')
df['Label'] = df['Label'].replace(4,'Fear')
df['Label'] = df['Label'].replace(5,'Surprise')
train_df, test_df = train_test_split(df, test_size=test_size_average, stratify=df['Label'], random_state=42)

### Conteo de labels

In [11]:
label_counts = train_df['Label'].value_counts()

### Conteo de palabras

In [12]:
word_counts = defaultdict(lambda: defaultdict(int))

# Iterar sobre los mensajes y las etiquetas
for index, row in train_df.iterrows():
    words = row['Text'].split()
    label = row['Label']
    for word in words:
        if len(word) >= 3:
            word_counts[word][label] += 1
            
# Convertir el diccionario a un DataFrame de pandas
model_df = pd.DataFrame(word_counts).fillna(0).astype(int).T

#Sumar un elemento para evitar probabilidades nulas
model_df += 1;

### Calculo de probablidades

In [13]:
# Duplicar columnas de etiquetas con sufijo "_proba"
for label in model_df.columns:
    model_df[label + 'P'] = model_df[label] / label_counts[label]

# Ordenar las columnas
model_df = model_df.sort_index(axis=1)

### Exportación

In [14]:
#Guardar modelo en un csv
model_df.to_csv('model.csv', index=True)

model_df.head()

,Anger,AngerP,Fear,FearP,Joy,JoyP,Love,LoveP,Sadness,SadnessP,Surprise,SurpriseP
just,4658,0.101583,3164,0.082892,8904,0.078899,2344,0.084795,9344,0.096381,1200,0.100184
look,547,0.011929,500,0.013099,1829,0.016207,355,0.012842,1289,0.013296,274,0.022875
myself,3069,0.066930,2107,0.055200,4844,0.042923,1142,0.041312,5543,0.057174,617,0.051511
and,26922,0.587124,22030,0.577155,69381,0.614791,18008,0.651449,56456,0.582327,7308,0.610119
feel,30323,0.661295,24728,0.647839,82108,0.727566,19637,0.710379,67268,0.693849,7952,0.663884
